## 🎯 1) Función de Pérdida (Loss), Optimizadores y Scheduler

La función de pérdida mide **qué tan mal está prediciendo tu modelo**. Es una métrica usada por el optimizador para ajustar los pesos.  

**Matemáticamente, buscamos minimizarla durante el entrenamiento.**

## 2) 📐 Fundamento Matemático


### 2.1) Cálculo de la función `nn.CrossEntropyLoss` paso a paso

Dado un vector de *logits* (valores sin normalizar):

```python
logits = torch.tensor([[1.0, 2.0, 0.1]])
labels = torch.tensor([1])  # Clase real
```

La función `nn.CrossEntropyLoss` aplica internamente la función **Softmax** para convertir los logits en probabilidades:

$$
\text{Softmax}(x_i) = \frac{e^{x_i}}{\sum_j e^{x_j}}
$$

Aplicando Softmax sobre los logits obtenemos:

$$
\text{Probabilidades} = [0.2424,\ 0.6590,\ 0.0986]
$$

La clase correcta (según la etiqueta) es la **clase 1**, con una probabilidad asignada de:

$$
p_{\text{correcta}} = 0.6590
$$

Entonces, la **función de pérdida Cross Entropy** se define como:

$$
\mathcal{L}(y, \hat{y}) = -\log(\hat{y}_{\text{clase correcta}})
$$

Aplicando los valores:

$$
\mathcal{L}(1, [0.2424, 0.6590, 0.0986]) = -\log(0.6590) \approx 0.4170
$$

👉 Esta pérdida será **menor cuanto más alta sea la probabilidad asignada a la clase correcta**, y **mayor si el modelo se equivoca o no está seguro**.

### 2.1) Implementación de Código

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [2]:
# Simulamos una predicción: logits para 3 clases
logits = torch.tensor([[1.0, 2.0, 0.1]])  # sin aplicar softmax
labels = torch.tensor([1])  # clase real

criterion = nn.CrossEntropyLoss()
loss = criterion(logits, labels)

print(f"CrossEntropyLoss: {loss.item():.4f}")
print("Probabilidades:", F.softmax(logits, dim=1))

CrossEntropyLoss: 0.4170
Probabilidades: tensor([[0.2424, 0.6590, 0.0986]])


## 3) 🧪 Ejemplos

### 3.1) ¿Qué es un Optimizador?

Un optimizador se encarga de **actualizar los pesos de la red** para minimizar la función de pérdida.  
Ejemplos populares: `SGD`, `Adam`, `RMSprop`.  
Utilizan el gradiente para decidir **cómo mover los pesos en cada paso** del entrenamiento.

### 3.2) ¿Qué es un Scheduler?

Un scheduler (programador de tasa de aprendizaje) ajusta dinámicamente el **learning rate**.  
Esto puede ayudar a converger mejor y más rápido, o evitar estancarse.  

Ejemplo: reducir el `lr` cuando el modelo deja de mejorar.

### 3.3) Implementación de Código de Optimizer + Scheduler

In [3]:
import torch.optim as optim
from torch.optim import lr_scheduler

In [4]:
model_ft = torch.nn.Linear(10, 2)

criterion = nn.CrossEntropyLoss()

# SGD con momentum
optimizer_ft = optim.SGD(model_ft.parameters(), lr=0.001, momentum=0.9)

# Scheduler: baja el LR cada 7 epochs
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1)

for epoch in range(15):
    optimizer_ft.step()
    exp_lr_scheduler.step()
    print(f"Epoch {epoch+1}: LR = {optimizer_ft.param_groups[0]['lr']}")


Epoch 1: LR = 0.001
Epoch 2: LR = 0.001
Epoch 3: LR = 0.001
Epoch 4: LR = 0.001
Epoch 5: LR = 0.001
Epoch 6: LR = 0.001
Epoch 7: LR = 0.0001
Epoch 8: LR = 0.0001
Epoch 9: LR = 0.0001
Epoch 10: LR = 0.0001
Epoch 11: LR = 0.0001
Epoch 12: LR = 0.0001
Epoch 13: LR = 0.0001
Epoch 14: LR = 1e-05
Epoch 15: LR = 1e-05


Lo que vemos es lo siguiente:
```
Epoch | LR
1–7   | 0.001
8–14  | 0.0001
15+   | 0.00001
```

## 4) 💡 Tips

- Elegí la función de pérdida adecuada: clasificación, regresión, etc.
- Monitoreá `train loss` vs `val loss`. Si `train ↓` pero `val ↑`, estás sobreajustando.
- Experimentá con diferentes optimizadores y programadores de LR.